In [2]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [3]:
import sys
import os
import h5py
import json
import scipy.io
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import Input
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [5]:
def Nuclei_feat(mag):
    nuc_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/nuc_Feature/'

    nuc_file_name = nuc_dir+ 'bach_mp_nuclei_feature_new_2.mat'
    nuc_data = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data.append(row_data)
                    
    Nuc_feat = np.array(nuc_data)
    del nuc_data
    
    Nuc_feat = Nuc_feat.reshape(len(Nuc_feat), 256,256,1)
    
    Nuc_feat_sc = nuc_scaler(Nuc_feat)
    
    return Nuc_feat_sc

In [6]:
def nuc_scaler(data):
    scaler = StandardScaler()
    data = data.reshape((len(data),256*256))
    scaler.fit(data)
    data_sc = scaler.transform(data)
    data_sc = data_sc.reshape((len(data),256, 256,1))
    return data_sc

In [7]:
def PCA_feat(mag):
    pca_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/PCA_Feature/'

    pca_file_name = pca_dir+ 'bach_pca_feature_new_2.mat'
    pca_data = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data.append(column)
                    
    PCA_data = np.array(pca_data)
    
    PCA_data_sc = pca_scaler(PCA_data)    
    del PCA_data
    
    return PCA_data_sc

In [8]:
def pca_scaler(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data_sc = scaler.transform(data)
    return data_sc

In [9]:
def VGG_feat(mag):
    vgg_BACH = np.load('fine_bottleneck_feat_bach_2_' + mag +'.npy')
    return vgg_BACH

In [10]:
def Load_Model(mag):    
    #model_name = 'Final_concat_Model_' + mag
    model_name = 'Final_concat_Model_t2_' + mag
    
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")
    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [14]:
def Load_Data(mag):
    bach_b_test, bach_m_test = 100, 200
    nb_bach_test =  bach_b_test + bach_m_test
    test_labels = np.array([0] * bach_b_test + [1] * bach_m_test)   
    
    Nuc_test = Nuclei_feat(mag)
    PCA_test = PCA_feat(mag)
    VGG_test = VGG_feat(mag)
    return Nuc_test,PCA_test,VGG_test, test_labels


In [35]:
mag_fact = '40'
loaded_model = Load_Model(mag_fact)
Nuc_bach,PCA_bach,VGG_bach, Labels_bach = Load_Data(mag_fact)
pred = loaded_model.predict(x = [Nuc_bach, PCA_bach, VGG_bach])    
accuracy = loaded_model.evaluate(x = [Nuc_bach, PCA_bach, VGG_bach], y = Labels_bach)
print('\n', 'BACH Test accuracy:', accuracy[1])

mag_fact = '100'
loaded_model = Load_Model(mag_fact)
Nuc_bach,PCA_bach,VGG_bach, Labels_bach = Load_Data(mag_fact)
pred = loaded_model.predict(x = [Nuc_bach, PCA_bach, VGG_bach])    
accuracy = loaded_model.evaluate(x = [Nuc_bach, PCA_bach, VGG_bach], y = Labels_bach)
print('\n', 'BACH Test accuracy:', accuracy[1])

mag_fact = '200'
loaded_model = Load_Model(mag_fact)
Nuc_bach,PCA_bach,VGG_bach, Labels_bach = Load_Data(mag_fact)
pred = loaded_model.predict(x = [Nuc_bach, PCA_bach, VGG_bach])    
accuracy = loaded_model.evaluate(x = [Nuc_bach, PCA_bach, VGG_bach], y = Labels_bach)
print('\n', 'BACH Test accuracy:', accuracy[1])

mag_fact = '400'
loaded_model = Load_Model(mag_fact)
Nuc_bach,PCA_bach,VGG_bach, Labels_bach = Load_Data(mag_fact)
pred = loaded_model.predict(x = [Nuc_bach, PCA_bach, VGG_bach])    
accuracy = loaded_model.evaluate(x = [Nuc_bach, PCA_bach, VGG_bach], y = Labels_bach)
print('\n', 'BACH Test accuracy:', accuracy[1])

Loaded model from disk
300/300 [==============================] - 0s 721us/step

 BACH Test accuracy: 0.6000000238418579
Loaded model from disk
300/300 [==============================] - 0s 717us/step

 BACH Test accuracy: 0.6166666746139526
Loaded model from disk
300/300 [==============================] - 0s 724us/step

 BACH Test accuracy: 0.5799999833106995
Loaded model from disk
300/300 [==============================] - 0s 727us/step

 BACH Test accuracy: 0.6266666650772095
